<a href="https://colab.research.google.com/github/yudiks/LLM-apps/blob/main/Yudi_Copy_of_CISD410_Module14_Langchain_QuickstartGuide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# USE LLAMA2 to Build Email Response Generator

Why Llama2, and not OpenAI? Because llama2 is free.


In [ ]:
!pip install -qq -U langchain
!pip install -qq -U transformers
!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1  xformers  --index-url https://download.pytorch.org/whl/cu118
!pip install -qq -U accelerate bitsandbytes einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 28.6 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.2/819.2 MB 980.1 kB/

## Imports

In [ ]:
# Prompts + Langchain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# pipeline
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

## Get Model and Tokenizer

In [ ]:
class GLOBAL:
    # LLMs
    model_name = 'llama2-7b' # llama2-7b, llama2-13b
    temperature = 0
    top_p = 0.95
    repetition_penalty = 1.15

In [ ]:
def get_model(model_name):
    print('\nDownloading model: ', model_name, '\n\n')

    if model_name == 'llama2-7b':
        model_repo = 'daryl149/llama-2-7b-chat-hf'

        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_4bit=True,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            trust_remote_code=True
            )

        max_len = 2048

    elif model_name == 'llama2-13b':
        model_repo = 'daryl149/llama-2-13b-chat-hf'

        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_4bit=True,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            trust_remote_code=True
            )

        max_len = 8192

    else:
        print("Not implemented model (tokenizer and backbone)")

    return tokenizer, model, max_len

In [ ]:
%%time

tokenizer, model, max_len = get_model(model_name = GLOBAL.model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

CPU times: user 21.8 s, sys: 29.9 s, total: 51.7 s
Wall time: 2min 22s


## Create a LangChain Pipeline

In [ ]:
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    max_length = max_len,
    temperature = GLOBAL.temperature,
    top_p = GLOBAL.top_p,
    repetition_penalty = GLOBAL.repetition_penalty
)

llm = HuggingFacePipeline(pipeline = pipe)

## Create a Prompt Template

In [ ]:

prompt_template = """\
Generate an email response in the same language as in the email was provided.
Summarize the email FIRST, then generate an email response at the end.
Format the email to be short, concise and friendly.

{context}

Email: {email_to_respond}
Generated Email Response:
"""


PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "email_to_respond"]
)

chain = LLMChain(llm=llm, prompt=PROMPT)


In [ ]:
email_to_respond = f"""
    Hi team,

    As we gear up for our next team outing, we'd love to hear your ideas and preferences! We want to make this outing truly enjoyable for everyone, and your input is vital.

    Could you please take a moment to respond to this email with your top three preferred outing ideas? Whether it's a hiking trip, a city tour, a picnic in the park, or something entirely different, we want to know what you'd enjoy the most.

    Looking forward to hearing your suggestions!

    Best regards,
    Country Manager
"""

user_context = f"""
A friendly reply mail to agree upon and suggest preferred outing places in Folsom, Ca, USA. eg. Lake Natoma. Please be specific.
Add Signatory from Yudi Sutanto at the end of email.
"""

# THIS IS WHERE THE MAGIC HAPPENS
print(
  chain.run({
    "context": user_context,
    "email_to_respond": email_to_respond })
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generate an email response in the same language as in the email was provided.
Summarize the email FIRST, then generate an email response at the end. 
Format the email to be short, concise and friendly.


A friendly reply mail to agree upon and suggest preferred outing places in Folsom, Ca, USA. eg. Lake Natoma. Please be specific.
Add Signatory from Yudi Sutanto at the end of email.


Email: 
    Hi team,

    As we gear up for our next team outing, we'd love to hear your ideas and preferences! We want to make this outing truly enjoyable for everyone, and your input is vital.

    Could you please take a moment to respond to this email with your top three preferred outing ideas? Whether it's a hiking trip, a city tour, a picnic in the park, or something entirely different, we want to know what you'd enjoy the most.

    Looking forward to hearing your suggestions!

    Best regards,
    Country Manager

Generated Email Response:
Hi Team,
Thanks for reaching out! I think a hike in Lake 